# 🏦 Кредитный скоринг — CatBoost + ROC-AUC

Этот ноутбук поможет вам построить модель кредитного скоринга, которая предсказывает вероятность дефолта клиента.

Мы будем использовать библиотеку `catboost`, кросс-валидацию, метрику `ROC-AUC` и визуализируем важность признаков.


In [5]:
# 📦 Импорт библиотек
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, Pool
import gc


## 📥 Загрузка данных
Предполагается, что у вас есть:
- `train_data_0.csv` — тренировочные признаки;
- `train_target.csv` — метки классов (`target`).

In [7]:
from pathlib import Path

print("📥 Загрузка файлов...")
files = sorted(Path(".").glob("train_data/train_data_*.pq"))
dfs = [pd.read_parquet(f) for f in files]
train_data = pd.concat(dfs, ignore_index=True)
del dfs
gc.collect()

📥 Загрузка файлов...


MemoryError: Unable to allocate 11.5 GiB for an array with shape (59, 26162717) and data type int64

In [ ]:
train_target = pd.read_csv("train_target.csv")

In [ ]:
print("🧹 Объединение и преобразование типов...")
train = train_data.merge(train_target, on="id", how="inner")
del train_data
gc.collect()

In [ ]:
for col in train.columns:
    if col.startswith("enc_col_"):
        train[col] = train[col].astype("category")


In [ ]:
print("📊 Сэмплирование данных...")
sample = train.sample(500_000, random_state=42)  # можно 100_000, если память ограничена
X = sample.drop(columns=["id", "rn", "target"])
y = sample["target"]
print(f"Размер сэмпла: {X.shape}")

## 🧹 Подготовка признаков и целевой переменной
- Удалим служебные колонки `id` и `rn`
- Разделим данные на `X` (признаки) и `y` (целевую переменную)

In [4]:
# Разделение признаков и таргета
X = train.drop(columns=["id", "rn", "target"])
y = train["target"]

MemoryError: Unable to allocate 11.1 GiB for an array with shape (57, 26162717) and data type int64

## 🔁 Кросс-валидация с CatBoost
- Используем `StratifiedKFold` для сбалансированного разбиения
- Метрика — `ROC-AUC`
- Учитываем дисбаланс классов через `class_weights`

In [4]:
# Настройка кросс-валидации
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
auc_scores = []

# Цикл по фолдам
for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = CatBoostClassifier(
        iterations=300,
        learning_rate=0.1,
        depth=6,
        eval_metric="AUC",
        verbose=0,
        random_state=42,
        class_weights=[1, 20]  # боремся с дисбалансом классов
    )
    model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=30)

    y_pred_proba = model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, y_pred_proba)
    auc_scores.append(auc)
    print(f"Fold {fold+1} AUC: {auc:.4f}")

print(f"\nСреднее значение ROC-AUC: {np.mean(auc_scores):.4f}")

Fold 1 AUC: 0.6118
Fold 2 AUC: 0.5961
Fold 3 AUC: 0.6055
Fold 4 AUC: 0.6303
Fold 5 AUC: 0.6434

Среднее значение ROC-AUC: 0.6174


## 📊 Важность признаков
Посмотрим, какие признаки оказались самыми важными для модели.

In [5]:
feature_importance = model.get_feature_importance(prettified=True)
feature_importance.head(10)

,Feature Id,Importances
0,enc_col_33,16.802706
1,enc_col_18,14.786971
2,enc_col_27,14.242087
3,enc_col_52,9.820812
4,enc_col_28,8.982757
5,enc_col_50,8.794864
6,enc_col_51,7.002267
7,enc_col_21,6.880314
8,enc_col_19,6.491009
9,enc_col_44,6.196213
